In [1]:
from vm.mnemonic import WProtectMnemonic
from vm.analysis import PEAnalysis, WProtectEmulator, WProtectTracer
import vm.obfuscation
import semantic.flow
from vm.rule_generator import generate_rules

_Load binary file and recover its instructions_

In [2]:
binary = WProtectEmulator("if_all/if_jz.wp.exe")
#binary = WProtectEmulator("if_all/if_jb.wp.exe")
#binary = WProtectEmulator("if_all/if_jo.wp.exe")
#binary = WProtectEmulator("if_all/if_js.wp.exe")
#binary = WProtectEmulator("if_all/if_jbe.wp.exe")
#binary = WProtectEmulator("if_all/if_jl.wp.exe")
#binary = WProtectEmulator("if_all/if_jle.wp.exe")
#binary = WProtectEmulator("if_all/if_jp.wp.exe")
binary.find()
instructions = binary.recover_mnemonics(binary.offset, WProtectMnemonic)[1]
WProtectMnemonic.consolidate_names(instructions)

[INFO]	PE loaded
[INFO]	reachable code analysis done


[INFO]	WProtect vm mnemonics found at offset 0x40a901


[INFO]	Mnemonics loaded


_Reconstruct the control flow and get continuous code blocks._

In [3]:
def callback(offset, instruction, args, off):
    control_flow.add_node(offset, instruction, args, off)

control_flow = semantic.flow.WProtectControlFlow(instructions)
instance = WProtectTracer(binary, instructions)

machine = [instance]
index = 0
while index < len(machine):
    try:
        tmp = machine[index].step(callback)
        machine += tmp
    except semantic.flow.PositionInstructionError as err:
        pass
    index += 1
    
res = control_flow.compile_blocks()

In [41]:
import semantic.translator as tr
reload(tr)

<module 'semantic.translator' from 'semantic/translator.py'>

In [42]:
"".join(map(lambda x: hex(ord(x))[2:].zfill(2),tr.translate_blocks(res)))

reg_0: .byte 0;reg_1: .byte 0;reg_2: .byte 0;reg_3: .byte 0;reg_4: .byte 0;reg_5: .byte 0;reg_6: .byte 0;reg_7: .byte 0;reg_8: .byte 0;reg_9: .byte 0;reg_10: .byte 0;reg_11: .byte 0;reg_12: .byte 0;reg_13: .byte 0;reg_14: .byte 0;reg_15: .byte 0;reg_16: .byte 0;reg_17: .byte 0;reg_18: .byte 0;reg_19: .byte 0;reg_20: .byte 0;reg_21: .byte 0;reg_22: .byte 0;reg_23: .byte 0;reg_24: .byte 0;reg_25: .byte 0;reg_26: .byte 0;reg_27: .byte 0;reg_28: .byte 0;reg_29: .byte 0;reg_30: .byte 0;reg_31: .byte 0;reg_32: .byte 0;reg_33: .byte 0;reg_34: .byte 0;reg_35: .byte 0;reg_36: .byte 0;reg_37: .byte 0;reg_38: .byte 0;reg_39: .byte 0;reg_40: .byte 0;reg_41: .byte 0;reg_42: .byte 0;reg_43: .byte 0;reg_44: .byte 0;reg_45: .byte 0;reg_46: .byte 0;reg_47: .byte 0;reg_48: .byte 0;reg_49: .byte 0;reg_50: .byte 0;reg_51: .byte 0;reg_52: .byte 0;reg_53: .byte 0;reg_54: .byte 0;reg_55: .byte 0;reg_56: .byte 0;reg_57: .byte 0;reg_58: .byte 0;reg_59: .byte 0;reg_60: .byte 0;reg_61: .byte 0;reg_62: .byte 0;re

'0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000c3'

In [25]:
res.keys()

[4240105L, 4235387L, 4240452L, 4240273L]